# Tutorial 3 - Benchmarks

In this tutorial, we will give you some hints on how to benchmark you homomorphic encryption application, and choose the best suited parameters.

We recommend checking out the other tutorials first:
- ['Tutorial 0 - Getting Started'](https://github.com/OpenMined/TenSEAL/blob/master/tutorials/Tutorial%200%20-%20Getting%20Started.ipynb).
- ['Tutorial 1: Training and Evaluation of Logistic Regression on Encrypted Data'](https://github.com/OpenMined/TenSEAL/blob/master/tutorials/Tutorial%201%20-%20Training%20and%20Evaluation%20of%20Logistic%20Regression%20on%20Encrypted%20Data.ipynb).

Authors:
- Bogdan Cebere - Twitter: [@bcebere](https://twitter.com/bcebere)

## Introduction



TenSEAL is a library for doing homomorphic encryption operations on tensors. It's built on top of [Microsoft SEAL](https://github.com/Microsoft/SEAL), a C++ library implementing the BFV and CKKS homomorphic encryption schemes.



## Homomorphic Encryption Parameters Review

#### The scaling factor(CKKS)
The first step of the CKKS scheme is encoding a vector of real numbers into a plaintext polynomial.


The scaling factor defines the encoding precision for the binary representation of the number. Intuitively, we are talking about binary precision as pictured below:


<img src="assets/floating_point.png" alt="ckks-high-level" width="400"/>

#### The polynomial modulus degree (poly_modulus_degree)

The polynomial modulus($N$ in the diagram) directly affects:
 - The number of coefficients in plaintext polynomials
 - The size of ciphertext elements
 - The computational performance of the scheme (bigger is worse)
 - The security level (bigger is better).

In TenSEAL, as in Microsoft SEAL, the degree of the polynomial modulus must be a power of 2 (e.g. $1024$, $2048$, $4096$, $8192$, $16384$, or $32768$).

#### The coefficient modulus sizes

The last parameter required for the scheme is a list of binary sizes.
Using this list, SEAL will generate a list of primes of those binary sizes, called the coefficient modulus($q$ in the diagram).

The coefficient modulus directly affects:
 - The size of ciphertext elements
 - The length of the list indicates the level of the scheme(or the number of encrypted multiplications supported).
 - The security level (bigger is worse).
 
In TenSEAL, as in Microsoft SEAL, each of the prime numbers in the coefficient modulus must be at most 60 bits and must be congruent to 1 modulo 2*poly_modulus_degree.

## Setup 

In [1]:
!pip install tabulate

import tenseal as ts
import tenseal.sealapi as seal
import random
import pickle
import numpy as np
import math
from IPython.display import HTML, display
import tabulate
import pytest
 
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

enc_type_str = {
    ts.ENCRYPTION_TYPE.SYMMETRIC : "symmetric", 
    ts.ENCRYPTION_TYPE.ASYMMETRIC : "asymmetric",
}

scheme_str = {
    ts.SCHEME_TYPE.CKKS : "ckks", 
    ts.SCHEME_TYPE.BFV : "bfv",
}

def decrypt(enc):
    return enc.decrypt()

## Context serialization


The TenSEAL context is required for defining the perfomance and security of your application.

In a client-server setup, it is required only on the initial handshake, as the ciphertexts can be linked with an existing context on deserialization.

Here we are benchmarking the size of the context, depending on the input parameters.

In [2]:
ctx_size_benchmarks = [["Encryption Type", "Scheme Type", "Polynomial modulus", "Coefficient modulus sizes", "Saved keys", "Context serialized size", ]]

for enc_type in [ts.ENCRYPTION_TYPE.SYMMETRIC, ts.ENCRYPTION_TYPE.ASYMMETRIC]:
    for (poly_mod, coeff_mod_bit_sizes) in [
        (8192, [40, 21, 21, 21, 21, 21, 21, 40]),
        (8192, [40, 20, 40]),
        (8192, [20, 20, 20]),
        (8192, [17, 17]),
        (4096, [40, 20, 40]),
        (4096, [30, 20, 30]),
        (4096, [20, 20, 20]),
        (4096, [19, 19, 19]),
        (4096, [18, 18, 18]),
        (4096, [18, 18]),
        (4096, [17, 17]),
        (2048, [20, 20]),
        (2048, [18, 18]),
        (2048, [16, 16]),
    ]:
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        context.generate_galois_keys()
        context.generate_relin_keys()
        
        ser = context.serialize(save_public_key=True, save_secret_key=True, save_galois_keys=True, save_relin_keys=True)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "all", convert_size(len(ser))])
        
        if enc_type is ts.ENCRYPTION_TYPE.ASYMMETRIC:
            ser = context.serialize(save_public_key=True, save_secret_key=False, save_galois_keys=False, save_relin_keys=False)
            ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "Public key", convert_size(len(ser))])
  
        ser = context.serialize(save_public_key=False, save_secret_key=True, save_galois_keys=False, save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "Secret key",  convert_size(len(ser))])
                                    
        ser = context.serialize(save_public_key=False, save_secret_key=False, save_galois_keys=True, save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "Galois keys",  convert_size(len(ser))])
                                                                      
        ser = context.serialize(save_public_key=False, save_secret_key=False, save_galois_keys=False, save_relin_keys=True)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "Relin keys",  convert_size(len(ser))])
  
        ser = context.serialize(save_public_key=False, save_secret_key=False, save_galois_keys=False, save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "none", convert_size(len(ser))])
        
    for (poly_mod, coeff_mod_bit_sizes) in [
        (8192, [40, 21, 21, 21, 21, 21, 21, 40]),
        (8192, [40, 21, 21, 21, 21, 21, 40]),
        (8192, [40, 21, 21, 21, 21, 40]),
        (8192, [40, 21, 21, 21, 40]),
        (8192, [40, 21, 21, 40]),
        (8192, [40, 20, 40]),
        (4096, [40, 20, 40]),
        (4096, [30, 20, 30]),
        (4096, [20, 20, 20]),
        (4096, [19, 19, 19]),
        (4096, [18, 18, 18]),
        (2048, [20, 20]),
    ]:
        context = ts.context(
            scheme=ts.SCHEME_TYPE.BFV,
            poly_modulus_degree=poly_mod,
            plain_modulus=786433,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        
        context.generate_galois_keys()
        context.generate_relin_keys()
        
        ser = context.serialize(save_public_key=True, save_secret_key=True, save_galois_keys=True, save_relin_keys=True)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "all", convert_size(len(ser))])
        
        if enc_type is ts.ENCRYPTION_TYPE.ASYMMETRIC:
            ser = context.serialize(save_public_key=True, save_secret_key=False, save_galois_keys=False, save_relin_keys=False)
            ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "Public key", convert_size(len(ser))])
  
        ser = context.serialize(save_public_key=False, save_secret_key=True, save_galois_keys=False, save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "Secret key",  convert_size(len(ser))])
                                    
        ser = context.serialize(save_public_key=False, save_secret_key=False, save_galois_keys=True, save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "Galois keys",  convert_size(len(ser))])
                                                                      
        ser = context.serialize(save_public_key=False, save_secret_key=False, save_galois_keys=False, save_relin_keys=True)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "Relin keys",  convert_size(len(ser))])
  
        ser = context.serialize(save_public_key=False, save_secret_key=False, save_galois_keys=False, save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "none", convert_size(len(ser))])
        

display(HTML(tabulate.tabulate(ctx_size_benchmarks, tablefmt='html')))

Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Saved keys,Context serialized size
symmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",all,263.37 KB
symmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",Secret key,263.37 KB
symmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",Galois keys,86.88 MB
symmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",Relin keys,3.63 MB
symmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",none,112.0 B
symmetric,ckks,8192,"[40, 20, 40]",all,124.88 KB
symmetric,ckks,8192,"[40, 20, 40]",Secret key,124.87 KB
symmetric,ckks,8192,"[40, 20, 40]",Galois keys,11.89 MB
symmetric,ckks,8192,"[40, 20, 40]",Relin keys,503.81 KB
symmetric,ckks,8192,"[40, 20, 40]",none,91.0 B


## Context serialization interpretation

Some key points here would be:
 - The symmetric encryption creates smaller contexts than the public key ones.
 - Decreasing the length of the coefficient modulus decreases the size of the context but also the depth of available multiplications.
 - Decreasing the coefficient modulus sizes reduces the context size, but impacts the precision as well (for CKKS).
 - Galois keys increase the context size only for public contexts (without the secret key). Send them only when you need to perform ciphertext rotations on the other end.
 - Relinearization keys increase the context size only for public contexts. Send them only when you need to perform multiplications on ciphertexts on the other end.
 - When we send the secret key, the Relinearization/Galois key can be regenerated on the other end without sending them.

## Ciphertext serialization

We next review how different parameters impact the ciphertext serialization.

The first observation here is that the symmetric/asymmetric encryption switch doesn't actually impact the size of the ciphertext, only of the context.

We will review the benchmarks only for the asymmetric scenario.

In [3]:
data = [random.uniform(-10, 10) for _ in range(10 ** 3)]
network_data = pickle.dumps(data)
print("Plain data size in bytes {}".format(convert_size(len(network_data))))

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Encryption Type", "Scheme Type", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Ciphertext serialized size", "Encryption increase ratio"]]


for (poly_mod, coeff_mod_bit_sizes, prec) in [
    (8192, [60, 40, 60], 40),
    (8192, [40, 21, 21, 21, 21, 21, 21, 40], 40),
    (8192, [40, 21, 21, 21, 21, 21, 21, 40], 21),
    (8192, [40, 20, 40], 40),
    (8192, [20, 20, 20], 38),
    (8192, [60, 60], 38),
    (8192, [40, 40], 38),
    (8192, [17, 17], 15),
    (4096, [40, 20, 40], 40),
    (4096, [30, 20, 30], 40),
    (4096, [20, 20, 20], 38),
    (4096, [19, 19, 19], 35),
    (4096, [18, 18, 18], 33),
    (4096, [30, 30], 25),
    (4096, [25, 25], 20),
    (4096, [18, 18], 16),
    (4096, [17, 17], 15),
    (2048, [20, 20], 18),
    (2048, [18, 18], 16),
    (2048, [16, 16], 14),
]:
    context = ts.context(
        scheme=ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=poly_mod,
        coeff_mod_bit_sizes=coeff_mod_bit_sizes,
        encryption_type=enc_type,
    )
    scale = 2 ** prec
    ckks_vec = ts.ckks_vector(context, data, scale)
 
    enc_network_data = ckks_vec.serialize()
    ct_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.CKKS], poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec),convert_size(len(enc_network_data)), round(len(enc_network_data) / len(network_data), 2)])
    
for (poly_mod, coeff_mod_bit_sizes) in [
    (8192, [40, 21, 21, 21, 21, 21, 21, 40]),
    (8192, [40, 21, 21, 21, 21, 21, 40]),
    (8192, [40, 21, 21, 21, 21, 40]),
    (8192, [40, 21, 21, 21, 40]),
    (8192, [40, 21, 21, 40]),
    (8192, [40, 20, 40]),
    (4096, [40, 20, 40]),
    (4096, [30, 20, 30]),
    (4096, [20, 20, 20]),
    (4096, [19, 19, 19]),
    (4096, [18, 18, 18]),
    (2048, [20, 20]),
]:
    context = ts.context(
        scheme=ts.SCHEME_TYPE.BFV,
        poly_modulus_degree=poly_mod,
        plain_modulus=786433,
        coeff_mod_bit_sizes=coeff_mod_bit_sizes,
        encryption_type=enc_type,
    )
    vec = ts.bfv_vector(context, data)
    enc_network_data = vec.serialize()
    ct_size_benchmarks.append([enc_type_str[enc_type], scheme_str[ts.SCHEME_TYPE.BFV], poly_mod, coeff_mod_bit_sizes, "-",convert_size(len(enc_network_data)), round(len(enc_network_data) / len(network_data), 2)])

    
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Plain data size in bytes 8.8 KB


Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Precision,Ciphertext serialized size,Encryption increase ratio
asymmetric,ckks,8192,"[60, 40, 60]",2**40,229.88 KB,26.11
asymmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**40,427.25 KB,48.53
asymmetric,ckks,8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**21,427.18 KB,48.52
asymmetric,ckks,8192,"[40, 20, 40]",2**40,153.3 KB,17.41
asymmetric,ckks,8192,"[20, 20, 20]",2**38,104.61 KB,11.88
asymmetric,ckks,8192,"[60, 60]",2**38,128.13 KB,14.55
asymmetric,ckks,8192,"[40, 40]",2**38,92.17 KB,10.47
asymmetric,ckks,8192,"[17, 17]",2**15,37.21 KB,4.23
asymmetric,ckks,4096,"[40, 20, 40]",2**40,79.03 KB,8.98
asymmetric,ckks,4096,"[30, 20, 30]",2**40,66.37 KB,7.54


### Ciphertext serialization interpretation

Some observations are:
 - The polynomial modulus increase results in a ciphertext increase.
 - The length of coefficient modulus sizes impacts the ciphertext size.
 - The values of the coefficient modulus sizes impact the ciphertext size, as well as the precision.
 - For a fixed set of polynomial modulus and coefficient modulus sizes, changing the precision doesn't affect the ciphertext size.

### Understanding the ciphertext precision impact


In [4]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]


for data_pow in [-1, 0, 1, 5, 11, 21, 41, 51]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (8192, [60, 40, 60], 40),
        (8192, [40, 21, 21, 21, 21, 21, 21, 40], 40),
        (8192, [40, 21, 21, 21, 21, 21, 21, 40], 21),
        (8192, [40, 20, 40], 40),
        (8192, [20, 20, 20], 38),
        (8192, [60, 60], 38),
        (8192, [40, 40], 38),
        (8192, [17, 17], 15),
        (4096, [40, 20, 40], 40),
        (4096, [30, 20, 30], 40),
        (4096, [20, 20, 20], 38),
        (4096, [19, 19, 19], 35),
        (4096, [18, 18, 18], 33),
        (4096, [30, 30], 25),
        (4096, [25, 25], 20),
        (4096, [18, 18], 16),
        (4096, [17, 17], 15),
        (2048, [20, 20], 18),
        (2048, [18, 18], 16),
        (2048, [16, 16], 14),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "encrypt", "encryption failed"])
            continue
    
        decrypted = decrypt(ckks_vec)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == data:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "encrypt", "decryption prec 2 ** {}".format(-dec_prec)])
                break
        ckks_sum = ckks_vec + ckks_vec
        decrypted = decrypt(ckks_sum)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] + data[0]]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "sum", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                

# We add more depth for the multiplication scenario
for data_pow in [-1, 0, 1, 5, 11, 21, 41, 51]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (8192, [60, 40, 40, 60], 40),
        (8192, [40, 21, 21, 40], 40),
        (8192, [40, 21, 21, 40], 21),
        (8192, [40, 20, 20, 40], 40),
        (8192, [20, 20, 20], 38),
        (4096, [40, 20, 40], 40),
        (4096, [30, 20, 30], 40),
        (4096, [20, 20, 20], 38),
        (4096, [19, 19, 19], 35),
        (4096, [18, 18, 18], 33),
        (4096, [30, 30, 30], 25),
        (4096, [25, 25, 25], 20),
        (4096, [18, 18, 18], 16),
        (2048, [18, 18, 18], 16),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            continue
                
        try:
            ckks_mul = ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-1 - 2^0],8192,"[60, 40, 60]",2**40,encrypt,decryption prec 2 ** -30
[2^-1 - 2^0],8192,"[60, 40, 60]",2**40,sum,decryption prec 2 ** -29
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**40,encrypt,decryption prec 2 ** -33
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**40,sum,decryption prec 2 ** -32
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**21,encrypt,decryption prec 2 ** -10
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**21,sum,decryption prec 2 ** -9
[2^-1 - 2^0],8192,"[40, 20, 40]",2**40,encrypt,decryption prec 2 ** -34
[2^-1 - 2^0],8192,"[40, 20, 40]",2**40,sum,decryption prec 2 ** -33
[2^-1 - 2^0],8192,"[20, 20, 20]",2**38,encrypt,decryption prec 2 ** -28
[2^-1 - 2^0],8192,"[60, 60]",2**38,encrypt,decryption prec 2 ** -27


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star TenSEAL on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star TenSEAL](https://github.com/OpenMined/TenSEAL)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org).

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)

## References

1. Yongsoo Song, Introduction to CKKS, [Private AI Bootcamp](microsoft.com/en-us/research/event/private-ai-bootcamp/#!videos).
2. [Microsoft SEAL](https://github.com/microsoft/SEAL).
3. Daniel Huynh, [CKKS Explained Series](https://blog.openmined.org/ckks-explained-part-1-simple-encoding-and-decoding/).